## Import

In [9]:
import rasterio
import numpy as np

from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LeakyReLU
#import tensorflow_hub as hub

from IAdeforestation.preprocessing import *
from IAdeforestation.tools import *
from IAdeforestation.training import *

In [2]:
#hub_layer = hub.KerasLayer("https://tfhub.dev/google/remote_sensing/eurosat-resnet50/1", tags=['train'], input_shape=(64,64,3))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [2]:
def split_predict_img(img, model, raster_template, mean, std, file_path='img', crop_size=64, split_size=171, img_max_val=65535):
    NB_CHANNEL = img.shape[0]
    TOTAL_WIDTH = img.shape[1]
    TOTAL_HEIGHT = img.shape[2]
    
    orignal_transform = raster_template.transform
    orignal_crs = raster_template.crs
    orignal_dtypes = raster_template.dtypes[0]
        
    new_img = np.array([]).reshape(0,split_size*crop_size)
    
    for i in range(split_size):
        
        batch = []
        for j in range(split_size):
            split_img = img[:,i*crop_size:(i+1)*crop_size,j*crop_size:(j+1)*crop_size]
            
            split_img = np.float32(np.moveaxis(split_img[:,:,:], 0, -1))
            
            split_img = normalize(split_img, mean, std)

            batch.append(split_img)
            
        batch = np.asarray(batch)
        y_pred = predicted_labels.append(model.predict(batch))
        y_pred = np.ones((1,171))
        
        new_horizontal_band = create_horizontal_band(y_pred)
        new_img = np.vstack([new_img, new_horizontal_band])
        
    return new_img

In [3]:
def create_horizontal_band(y_pred, nb_image=171, img_size=64, img_maxval=65535):
    new_band = np.array([]).reshape(img_size,0)
    
    for y in y_pred:
        new_band = np.hstack([new_band, (np.ones((64,64)) * y *img_maxval)])
    
    return new_band

In [7]:
SAVED_MODELS_PATH = os.path.join("saved_models","transfer_learning", "Coffee")
MODEL_NAME = 'new1'

model = load_model(os.path.join(SAVED_MODELS_PATH, 'final_cv_coffee_fall_1', 'final_cv_coffee_fall_1_2.h5'), custom_objects={'LeakyReLU':LeakyReLU})

In [8]:
SENTINEL_IMAGES_PATH = 'SentinelImages'

paths = os.listdir(SENTINEL_IMAGES_PATH)
paths

['a', 'b', 'c', 'd', 'e']

In [ ]:
SENTINEL_IMAGES_PATH = 'SentinelImages'
PREDICTED_PATH = 'PredictedImages'

for sentinel_image_path in paths[0:1]:
    print(sentinel_image_path)
    raster_paths = get_raster_paths(os.path.join(SENTINEL_IMAGES_PATH, sentinel_image_path))
    raster_dict = load_raster_img(raster_paths)
    image_dict = resample_bands(raster_dict)
    
    l = list(image_dict.values())
    orignial_img = np.asarray(l)
    
    final_img = np.asarray(split_predict_img(orignial_img, model, raster_dict['B02'], mean=eurosat_params['mean'], std=eurosat_params['std']))
    
    with rasterio.open(os.path.join(PREDICTED_PATH, sentinel_image_path + '_pred', name+'.tiff'), 'w', driver='Gtiff',
                                                  width=final_img.shape[1],
                                                  height=final_img.shape[2],
                                                  count=final_img.shape[0],
                                                    dtype=raster_dict['B02'].dtypes[0],
                                                   crs=raster_dict['B02'].crs,
                                                   transform=raster_dict['B02'].transform) as dst :
            dst.write(final_img)

Fall\a
